%run ../ipynb_util_tars.py

In [1]:
import os

from qdrant_client import QdrantClient, models

client = QdrantClient(
    os.getenv("QDRANT_API_URL"),
    port=os.getenv("QDRANT_API_PORT"),
    api_key=os.getenv("QDRANT_API_KEY"),
)

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


In [42]:
# scroll over points that HAVE labels ("labels" field is not null)
scroll_result = client.scroll(
    collection_name="publications",
    limit=500,
    with_payload=["id", "labels", "xai", "description"],
    scroll_filter=models.Filter(
        must_not=[
            models.IsNullCondition(
                is_null=models.PayloadField(key="labels")
            )
        ]
    )
)
print(len(scroll_result[0]))

for point in scroll_result[0]:
    if len(point.payload["xai"]) != 1:
        print("huh", point.payload["id"])

    shap_values = point.payload["xai"][0]["xai_values"]["token_scores"]
    # check if each value is in the range [-1, 1]
    # the shap_values are of size [len(tokens), 17]
    for token_scores in shap_values:
        for score in token_scores:
            if score < -0.1 or score > 0.1:
                print(point.payload["id"])
                break

print()
res = scroll_result[0][6]
print(res.payload["id"])
print(res.payload["labels"])
print(res.payload["xai"][0]["xai_method"])
print(res.payload["xai"][0]["predicted_label"])
print(res.payload["xai"][0]["probs"])
print(res.payload["xai"][1]["xai_method"])
print(res.payload["xai"][1]["predicted_label"])
print(res.payload["xai"][1]["probs"])
print()
print(res.payload["xai"][0]["xai_values"]["input_tokens"])
print(res.payload["xai"][1]["xai_values"]["input_tokens"])
#print(res.payload["xai"][2]["xai_values"]["input_tokens"])
print()
print(res.payload["xai"][0]["xai_values"]["token_scores"])
print(res.payload["xai"][1]["xai_values"]["token_scores"])
#print(res.payload["xai"][2]["xai_values"]["token_scores"])
print(res.payload["description"])

384
huh oai:www.zora.uzh.ch:62390
huh oai:www.zora.uzh.ch:85053
huh oai:www.zora.uzh.ch:85506
huh oai:www.zora.uzh.ch:86456
huh oai:www.zora.uzh.ch:87012
huh oai:www.zora.uzh.ch:87341
huh oai:www.zora.uzh.ch:88401
huh oai:www.zora.uzh.ch:95626
huh oai:www.zora.uzh.ch:95628
huh oai:www.zora.uzh.ch:95931
huh oai:www.zora.uzh.ch:97395
huh oai:www.zora.uzh.ch:97908
huh oai:www.zora.uzh.ch:98340
huh oai:www.zora.uzh.ch:99291
huh oai:www.zora.uzh.ch:99546
huh oai:www.zora.uzh.ch:99708
huh oai:www.zora.uzh.ch:99859
huh oai:www.zora.uzh.ch:100312
huh oai:www.zora.uzh.ch:101518
huh oai:www.zora.uzh.ch:101521
huh oai:www.zora.uzh.ch:101669
huh oai:www.zora.uzh.ch:102805
huh oai:www.zora.uzh.ch:104443
huh oai:www.zora.uzh.ch:104474
huh oai:www.zora.uzh.ch:106906
huh oai:www.zora.uzh.ch:107535
huh oai:www.zora.uzh.ch:110372
huh oai:www.zora.uzh.ch:110487
huh oai:www.zora.uzh.ch:110767
huh oai:www.zora.uzh.ch:111097
huh oai:www.zora.uzh.ch:111309
huh oai:www.zora.uzh.ch:111494
huh oai:www.zora.uzh.

In [43]:
# check if all 384 points contain all xai methods and only them
xai_methods = set(["shap-partition", "shap-partition-tfidf"])
c = 0
for point in scroll_result[0]:
    xai_methods_for_this_point = xai_methods.copy()
    #print()
    #print(len(point.payload["xai"]))
    # print the method names in the xai field
    #print([x["xai_method"] for x in point.payload["xai"]])
    for xai in point.payload["xai"]:
        #print(xai_methods_for_this_point)
        if xai["xai_method"] not in xai_methods_for_this_point:
            print("missing", xai["xai_method"], point.id)
            c += 1
        assert xai["xai_method"] in xai_methods_for_this_point
        xai_methods_for_this_point.remove(xai["xai_method"])
        #print(xai["xai_method"], point.id)

    #assert len(xai_methods_for_this_point) == 0
print(c)


2
['shap-partition', 'shap-partition-tfidf']
{'shap-partition-tfidf', 'shap-partition'}
shap-partition 62390
{'shap-partition-tfidf'}
shap-partition-tfidf 62390

2
['shap-partition', 'shap-partition-tfidf']
{'shap-partition-tfidf', 'shap-partition'}
shap-partition 85053
{'shap-partition-tfidf'}
shap-partition-tfidf 85053

2
['shap-partition', 'shap-partition-tfidf']
{'shap-partition-tfidf', 'shap-partition'}
shap-partition 85506
{'shap-partition-tfidf'}
shap-partition-tfidf 85506

2
['shap-partition', 'shap-partition-tfidf']
{'shap-partition-tfidf', 'shap-partition'}
shap-partition 86456
{'shap-partition-tfidf'}
shap-partition-tfidf 86456

2
['shap-partition', 'shap-partition-tfidf']
{'shap-partition-tfidf', 'shap-partition'}
shap-partition 87012
{'shap-partition-tfidf'}
shap-partition-tfidf 87012

2
['shap-partition', 'shap-partition-tfidf']
{'shap-partition-tfidf', 'shap-partition'}
shap-partition 87341
{'shap-partition-tfidf'}
shap-partition-tfidf 87341

2
['shap-partition', 'shap-

In [19]:
xai_out_dict = []

for point in scroll_result[0]:
    xai_out_dict.append(
        {
            "id": point.payload["id"],
            "xai": point.payload["xai"],
        }
    )

In [ ]:
# save xai_out_dict on disk as json
# import json

# with open("xai_out_dict.json", "w") as f:
#    json.dump(xai_out_dict, f)


In [ ]:
points_with_labels_count = client.count(
    collection_name="publications",
    count_filter=models.Filter(
        must_not=[
            models.IsNullCondition(
                is_null=models.PayloadField(key="labels")
            )
        ]
    ),
    exact=True
)
print(points_with_labels_count)

sdgs = [f"sdg{i}" for i in range(1, 18)]
sdg_counts = {}

for sdg in sdgs:
    result = client.count(
        collection_name="publications",
        count_filter=models.Filter(
            must_not=[
                models.IsEmptyCondition(
                    is_empty=models.PayloadField(key=f"labels.{sdg}")
                )
            ]
        ),
        exact=True
    )
    sdg_counts[sdg] = result.count

print(sdg_counts)
print(sum(sdg_counts.values()))

count=384
{'sdg1': 6, 'sdg2': 13, 'sdg3': 43, 'sdg4': 2, 'sdg5': 22, 'sdg6': 3, 'sdg7': 17, 'sdg8': 26, 'sdg9': 13, 'sdg10': 31, 'sdg11': 3, 'sdg12': 30, 'sdg13': 41, 'sdg14': 14, 'sdg15': 79, 'sdg16': 34, 'sdg17': 7}
384


In [ ]:
assert sum(sdg_counts.values()) == points_with_labels_count.count

In [ ]:
client.count(
    collection_name="publications",
    count_filter=models.Filter(
        must_not=[
            models.IsEmptyCondition(
                is_empty=models.PayloadField(key=f"xai")
            )
        ]
    ),
    exact=True
)

CountResult(count=384)